Após limparmos nossos dados e removermos as inconsistências, podemos então voltar nossas atenções ao processo de análise propriamente dito, onde iremos manipular nossos dados de forma a responder as questões que objetivamos responder, sendo elas:

- Preço médio de venda de cada combustível por estado;
- Preço médio de venda da gasolina em Alegrete - RS;

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('datasets/cleaned_fuel.csv', sep=';', index_col=0)

In [3]:
alegrete_fuel = df[(df['Município'] == 'ALEGRETE') & (df['Produto'] == 'GASOLINA')] \
            .sort_values(by='Data da Coleta') \
            .filter(items=['Data da Coleta', "Valor de Venda"]) \
            .set_index(['Data da Coleta'])

alegrete_fuel.index = pd.to_datetime(alegrete_fuel.index)
alegrete_fuel = alegrete_fuel.groupby('Data da Coleta').agg({'Valor de Venda': np.average})
alegrete_fuel.to_csv('datasets/alegrete_fuel.csv', sep=';')

In [4]:
price_by_state = df.filter(items=['Estado - Sigla', 'Produto', 'Valor de Venda'])
price_by_state.shape

(496226, 3)

In [6]:
states = [x for x in price_by_state['Estado - Sigla'].unique()]
products = [x for x in price_by_state['Produto'].unique()]
prices = []

for state in states:
    for product in products:
        prices.append([state, product, price_by_state[(price_by_state['Estado - Sigla'] == state) & (price_by_state['Produto'] == product)].mean().values[0]])

del price_by_state

In [5]:
price_by_state.shape

(496226, 3)

In [28]:
price_by_state = pd.DataFrame({'Estado - Sigla': [x[0] for x in prices], 'Produto': [x[1] for x in prices], 'Preço Médio': [x[2] for x in prices]})
price_by_state['Preço Médio'] = price_by_state['Preço Médio'].apply(lambda x: round(x, 2))
price_by_state['Preço Médio'].fillna(0, inplace=True)
price_by_state = price_by_state.pivot("Estado - Sigla", "Produto", "Preço Médio")
price_by_state.to_csv('datasets/price_by_state.csv', sep=';')